
# **Asignatura de Big Data Analytics**

Prof. José M. Luna

jmluna@uco.es

---

En primer lugar, debemos instalar todas las dependencias necesarias para poder ejecutar el algoritmo. Así, tenemos que instalar Java 8, Spark 3.0 con Hadoop 3.2, y findspark para alojar spark en el sistema (acceder a Spark desde un Notebook).

In [ ]:
# Instalacion de java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Instalacion de spark
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# Descomprimimos el fichero de spark
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# Instalamos findspark
!pip install -q findspark

# Establecemos las variables de entorno para Java y Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

Ejecutamos una sesion local de spark para comprobar que esta funcionando.

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Importamos el conjunto de datos con el que queremos trabajar y comprobamos que se ha subido correctamente

In [ ]:
from google.colab import files
uploaded = files.upload()

!ls

Saving iris.csv to iris.csv
iris.csv  sample_data  spark-3.0.0-bin-hadoop3.2  spark-3.0.0-bin-hadoop3.2.tgz


Leemos el dataset con el que queremos trabajar, en este caso vamos a trabajar con el dataset de Iris



In [ ]:
dataset = spark.read.csv('iris.csv',inferSchema=True, header =True)

Mostramos los atributos que tiene el dataset y su tipo

In [ ]:
dataset.printSchema()

root
 |-- sepallength: double (nullable = true)
 |-- sepalwidth: double (nullable = true)
 |-- petallength: double (nullable = true)
 |-- petalwidth: double (nullable = true)
 |-- variety: string (nullable = true)



Ahora vamos a trabajar con todas las variables y la varible variety será la salida. Creamos un conjunto de datos donde los inputs serán un vector

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

df = VectorAssembler(inputCols=['sepallength', 'sepalwidth', 'petallength', 'petalwidth'], outputCol = 'inputs').transform(dataset)

df = df.select(['inputs','variety'])

df.printSchema()

labelIndexer = StringIndexer(inputCol='variety', outputCol='Class').fit(df)
df = labelIndexer.transform(df)

root
 |-- inputs: vector (nullable = true)
 |-- variety: string (nullable = true)



Dividimos el conjunto de datos en train y test. Vamos a considerar una partición del tipo 80/20

In [ ]:
trainData,testData = df.randomSplit([0.8,0.2])

Indicamos qué atributos serán las entradas y cuales serán la salida. Finalmente, entrenamos y testeamos con trainData y testData, y mostramos las predicciones

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'inputs', labelCol = 'Class', maxDepth = 3)

#Entrenamos el modelo con el conjunto de train
model = dt.fit(trainData)

#Hacemos predicciones
predictions = model.transform(testData)

#Mostramos las predicciones
predictions.select('Class', 'prediction').show()

+-----+----------+
|Class|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  2.0|       2.0|
|  2.0|       2.0|
|  2.0|       2.0|
+-----+----------+
only showing top 20 rows



Mostramos resultados del modelo en test

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="Class", predictionCol="prediction", metricName = "accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))
print("Accuracy = %g " % accuracy)

Test Error = 0.0571429
Accuracy = 0.942857 


Mostramos el arbol resultado del modelo

In [ ]:
print(model.toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_cc359943ea15, depth=3, numNodes=7, numClasses=3, numFeatures=4
  If (feature 2 <= 2.45)
   Predict: 0.0
  Else (feature 2 > 2.45)
   If (feature 3 <= 1.75)
    If (feature 2 <= 5.05)
     Predict: 1.0
    Else (feature 2 > 5.05)
     Predict: 2.0
   Else (feature 3 > 1.75)
    Predict: 2.0

